# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time


In [2]:
url = "https://books.toscrape.com/catalogue/"

def get_book_url(url,page_nmbr = 5):
    all_books_url =[]
    for i in range(page_nmbr):
        bk_lst_page_url = url+ 'category/books_1/page-'+ str(i+1)+'.html'
        response = requests.get(bk_lst_page_url )
        soup = BeautifulSoup(response.content)

        tbls = soup.find_all('article', attrs={'class': 'product_pod'})
        # print(tbls)
        for line in tbls:
            # print(line)
            bk_link_tmp = line.find('div', attrs={'class':'image_container'}).find('a')['href']
            bk_link = str(urljoin(bk_lst_page_url, bk_link_tmp))
            all_books_url.append(bk_link)
        time.sleep(2)
    return all_books_url


def get_bk_details(book_url_list):
    all_books = []
    for bk_url in book_url_list:
        bk_details = BeautifulSoup(requests.get(bk_url).content)
        # print(bk_details)
        bk_price = bk_details.find('th', string='Price (incl. tax)').find_next_sibling('td').get_text().strip('£')
        bk_UPC = bk_details.find('th', string='UPC').find_next_sibling('td').get_text().strip('')
        bk_genre = bk_details.find('ul', attrs={'class':'breadcrumb'}).find_all('li')[2].find('a').get_text()

        bk_Availability_tmp = str(bk_details.find('th', string='Availability').find_next_sibling('td').get_text().strip(''))

        if 'In stock' in bk_Availability_tmp:
            bk_Availability = 'yes'
        else:
            bk_Availability = 'no'
        
        bk_description = bk_details.find('meta', attrs={'name':'description'})['content'].strip('')
        
        bk_review = str(bk_details.find('div', attrs={'class':'col-sm-6 product_main'})\
            .find('p', class_='star-rating')['class'][1])
        
        bk_title = bk_details.find('div', attrs={'class':'col-sm-6 product_main'})\
            .find('h1').get_text()

        book_record = {
            'UPC': bk_UPC,
            'Title': bk_title,
            'Price (£)': bk_price,
            'Rating': bk_review,
            'Genre': bk_genre,
            'Availability': bk_Availability,
            'Description': bk_description,
            'href': bk_url 
        }
        # print(bk_url, bk_title, bk_price, bk_UPC,bk_Availability, bk_review, bk_genre) #bk_description, 
        all_books.append(book_record)
        time.sleep(0.5)
    return all_books

def filter_book(rating_v, price_v, pages=2):
    # urls = []
    # all_book_json = []
    urls = get_book_url(url,pages)
    print(len(urls))
    all_book_json = get_bk_details(urls)
    df = pd.DataFrame(all_book_json)
    review_map = {'One':1,'Two':2,'Three':3,'Four':4, 'Five':5}
    df['Rating']=df['Rating'].replace(review_map).astype(int)
    df['Price (£)']=df['Price (£)'].astype(float)

    df_filtered = df[(df['Rating']>=rating_v) & (df['Price (£)']>=price_v)]
    return df_filtered


In [ ]:
pd.set_option('display.max_colwidth', None)

final_df = filter_book(rating_v=3, price_v = 25, pages = 1)
print(final_df)

20
                 UPC  \
0   a897fe39b1053632   
3   e00eb4fd7b871a48   
4   4165285e1663650f   
6   2597b5a345f45e1b   
13  3b1c02bac2a429e6   
14  a34ba96d4081e6a4   
15  deda3e61b9514b83   

                                                                                 Title  \
0                                                                 A Light in the Attic   
3                                                                        Sharp Objects   
4                                                Sapiens: A Brief History of Humankind   
6                                   The Dirty Little Secrets of Getting Your Dream Job   
13                             Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)   
14                                                           Rip it Up and Start Again   
15  Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991   

    Price (£)  Rating           Genre Availability  \
0       51.77       3        

/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_90405/2618980314.py:68: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Rating']=df['Rating'].replace(review_map).astype(int)


#### Test Code

In [4]:


# url = "https://books.toscrape.com/catalogue/"
# bk_lst_url = url+ 'category/books_1/page-'+ '1'+'.html' #'/catalogue/category/books_1/page-5.html'
# print(bk_lst_url)
# review_map = {'One':1,'Two':2,'Three':3,'Four':4, 'Five':5}

# response = requests.get(bk_lst_url )
# # response = requests.get(url + '/catalogue/category/books_1/page-'+ '1'+'.html')
# soup = BeautifulSoup(response.content)
# tbls = soup.find_all('article', attrs={'class': 'product_pod'})
# df = pd.DataFrame(columns=['UPC','Title','Price (£)','Rating','Genre','Availability','Description'])
# all_books = []
# for line in tbls:
#     bk_link_tmp = [x['href'] for x in line.select('h3 a')][0]
#     bk_link = urljoin(bk_lst_url, bk_link_tmp)
#     # bk_title = [x['title'] for x in line.select('h3 a')][0]
#     bk_details = BeautifulSoup(requests.get(bk_link).content)
#     # print(bk_details)
#     bk_price = bk_details.find('th', string='Price (incl. tax)').find_next_sibling('td').get_text().strip('£')
#     bk_UPC = bk_details.find('th', string='UPC').find_next_sibling('td').get_text().strip('')
#     bk_genre = bk_details.find('ul', attrs={'class':'breadcrumb'}).find_all('li')[2].find('a').get_text()

#     bk_Availability_tmp = str(bk_details.find('th', string='Availability').find_next_sibling('td').get_text().strip(''))

#     if 'In stock' in bk_Availability_tmp:
#         bk_Availability = 'yes'
#     else:
#         bk_Availability = 'no'
    
#     # bk_description = bk_details.find('article', attrs={'class': 'product_page'})\
#     #     .find('div', attrs={'id':'product_description'})\
#     #         .find_next('p').get_text().strip()

#     desc = bk_details.find('meta', attrs={'name':'description'})['content'].strip('')
#     print(desc)
#     bk_review = str(bk_details.find('div', attrs={'class':'col-sm-6 product_main'})\
#         .find('p', class_='star-rating')['class'][1])
    
#     bk_title = bk_details.find('div', attrs={'class':'col-sm-6 product_main'})\
#         .find('h1').get_text()

#     book_record = {
#         'UPC': bk_UPC,
#         'Title': bk_title,
#         'Price (£)': bk_price,
#         'Rating': bk_review,
#         'Genre': bk_genre,
#         'Availability': bk_Availability,
#         'Description': bk_description,
#         'href': bk_link 
#     }
#     print(bk_link, bk_title, bk_price, bk_UPC,bk_Availability, bk_review, bk_genre) #bk_description, 
#     all_books.append(book_record)
#     break
